#### Aquí encontrarás todo lo que necesitas saber: https://www.kaggle.com/competitions/desafio-de-prediccion-de-precios-de-portatiles

## Librerias

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager
import bootcampviztools as bt
import statsmodels.api as sm
import warnings
import sklearn
import os
import cv2
import re


from matplotlib import style
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif, SelectFromModel, RFE, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression, LogisticRegression, ElasticNet
from sklearn.metrics import classification_report, ConfusionMatrixDisplay,  mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, silhouette_score, silhouette_samples, mutual_info_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder, scale, FunctionTransformer, OneHotEncoder
from matplotlib.ticker import FixedLocator, FixedFormatter
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.utils import resample
from scipy import stats
from scipy.stats import mannwhitneyu

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor


warnings.filterwarnings('ignore')
style.use('ggplot') or plt.style.use('ggplot')

## Data

In [2]:
### Para que funcione necesitas bajarte los archivos de datos de Kaggle 
pcs = pd.read_csv("./data/train.csv")

In [3]:
target = "Price_euros"

In [4]:
var_desc = ["laptop_ID", "Product"] 
pcs = pcs.drop(["laptop_ID", "Product"], axis=1) 

X = pcs.drop("Price_euros", axis=1) 
y = pcs["Price_euros"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [5]:
X_train['Inches'] = X_train['Inches'].astype(float)

In [6]:
X_train['Weight'] = X_train['Weight'].apply(lambda x: float(x.replace('kg', '').replace(',', '.')))

In [7]:
X_train['Ram'] = X_train['Ram'].apply(lambda x: int(x.replace('GB', '')))

In [8]:
marcas_ppales = X_train['Company'].value_counts().sort_values(ascending=False) / X_train['Company'].count()
top_10_companies = marcas_ppales[marcas_ppales >= 0.01].index.tolist()
otras_marcas = [company for company in pcs['Company'].unique() if company not in top_10_companies]
X_train['Company'] = X_train['Company'].replace(otras_marcas, "Otra")

In [9]:
def extract_screen_resolution(df):
    def extract_resolution(screen_resolution):
        # Definir el patrón para buscar números de 3 a 4 dígitos separados por 'x'
        pattern = r'\b(\d{3,4})\s*x\s*(\d{3,4})\b'
        match = re.search(pattern, screen_resolution)
        if match:
            width = int(match.group(1))
            height = int(match.group(2))
            # Calcular el producto de anchura y altura
            resolution = width * height
            return resolution
        else:
            return 1

    # Definir las palabras clave para buscar en la descripción de la pantalla
    labels = ['Touchscreen', 'FullHD', 'Retina', 'HD+', '4K', 'IPS Panel']

    # Función para crear las columnas de características específicas de la pantalla
    def extract_screen_features(screen_resolution, labels):
        features = [1 if label in screen_resolution else 0 for label in labels]
        return features

    # Aplicar la función de extracción de resolución de pantalla
    df['Resolution'] = df['ScreenResolution'].apply(extract_resolution)

    # Aplicar la función de extracción de características específicas de la pantalla
    df[labels] = pd.DataFrame(df['ScreenResolution'].apply(lambda x: extract_screen_features(x, labels)).tolist(), index=df.index)

    return df

# Aplicar la función a tu DataFrame
X_train = extract_screen_resolution(X_train)

In [10]:
# Transformar la columna 'Cpu'
def extract_cpu_info(cpu):
    brand_pattern = r'(Intel|AMD)'
    speed_pattern = r'(\d+\.\d+)GHz'
    model_pattern = r'\b(?:Core [iI]\d+|Ryzen \d+)\b'
    series_pattern = r'(?:Intel|AMD)(.*?)(?:\b(?:Core [iI]\d+|Ryzen \d+|$)|\s\d+\.\d+GHz)'

    try:
        brand_match = re.search(brand_pattern, cpu)
        brand = brand_match.group(1) if brand_match else "Desconocida"

        speed_match = re.search(speed_pattern, cpu)
        speed = float(speed_match.group(1)) if speed_match else 1

        model_match = re.search(model_pattern, cpu)
        model = model_match.group() if model_match else "Desconocido"

        series_match = re.search(series_pattern, cpu)
        series = series_match.group(1).strip() if series_match else "Desconocida"

        return brand, speed, model, series
    except Exception as e:
        print("Error durante la extracción de información de la CPU:", e)
        return "Desconocida", -1.0, "Desconocido", "Desconocida"

# Aplicar la función extract_cpu_info a cada fila del DataFrame
for index, row in X_train.iterrows():
    cpu_info = extract_cpu_info(row['Cpu'])
    
    # Asignar los valores extraídos a las nuevas columnas en la misma fila
    X_train.at[index, 'CPU_marca'] = cpu_info[0]
    X_train.at[index, 'CPU_velocidad'] = cpu_info[1]
    X_train.at[index, 'CPU_modelo'] = cpu_info[2]
    X_train.at[index, 'CPU_serie'] = cpu_info[3]

In [11]:
# Transformar la columna 'Memory'
def extract_memory_info(memory):
    memory_pattern = r'(\d+)\s*(?:GB|T|TB)'
    keyword_pattern = r'HDD|SSD|Flash Storage|Hybrid'

    match = re.findall(memory_pattern, memory)
    if match:
        memory_value = int(match[0][0]) * 1024 if 'T' in memory else int(match[0][0])
    else:
        memory_value = 0

    keywords = re.findall(keyword_pattern, memory)
    storage_dict = {'HDD': 0, 'SSD': 0, 'Flash Storage': 0, 'Hybrid': 0}
    for keyword in keywords:
        storage_dict[keyword] = memory_value

    return storage_dict['HDD'], storage_dict['SSD'], storage_dict['Flash Storage'], storage_dict['Hybrid']

X_train['Memo_HDD'], X_train['Memo_SSD'], X_train['Memo_Flash'], X_train['Memo_Hybrid'] = zip(*X_train['Memory'].apply(extract_memory_info))
X_train.rename(columns={'Memo_HDD': 'Memo_HDD_GB', 'Memo_SSD': 'Memo_SSD_GB', 'Memo_Flash': 'Memo_Flash_GB', 'Memo_Hybrid': 'Memo_Hybrid_GB'}, inplace=True)


In [12]:
numeric_cols = X_train.select_dtypes(include=np.number)

# Calcular el rango intercuartílico solo para las columnas numéricas
Q1 = numeric_cols.quantile(0.25)
Q3 = numeric_cols.quantile(0.75)
IQR = Q3 - Q1

# Definir un límite para considerar outliers solo para las columnas numéricas
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Eliminar outliers solo para las columnas numéricas
pcs_eda_no_outliers = X_train[(numeric_cols >= lower_bound) & (numeric_cols <= upper_bound)].dropna()

In [13]:
var_fuera = []
cardinality = X_train.select_dtypes(include='object').nunique()

# Definir un umbral para la cardinalidad (por ejemplo, 10)
umbral_cardinalidad = 15

# Identificar las columnas categóricas con una cardinalidad alta
columnas_alta_cardinalidad = cardinality[cardinality > umbral_cardinalidad].index.tolist()

# Mover las columnas con alta cardinalidad a var_fuera y eliminarlas del DataFrame principal
var_fuera.append(columnas_alta_cardinalidad)
X_train.drop(columns=columnas_alta_cardinalidad, inplace=True)

In [14]:
columnas_categoricas = X_train.select_dtypes(include='object').columns

# Seleccionar solo las columnas numéricas que quedan en pcs_eda
columnas_numericas = X_train.select_dtypes(include=['int', 'float']).columns

# Aplicar get_dummies a las columnas categóricas seleccionadas
X_train_encoded = pd.get_dummies(X_train, columns=columnas_categoricas, drop_first=True)

In [15]:
# Crear un objeto StandardScaler
scaler = StandardScaler()

# Escalar las columnas numéricas seleccionadas en X_train_encoded
X_train_encoded[columnas_numericas] = scaler.fit_transform(X_train_encoded[columnas_numericas])

In [16]:
model_names = ["Regresion Lineal","DecisionTree","Random Forest","XGBoost","LightGBM","CatBoost"]
lr_reg = LinearRegression()
tree_reg = DecisionTreeRegressor(random_state= 42)
rf_reg = RandomForestRegressor(random_state= 42)
xgb_reg = XGBRegressor(random_state = 42)
lgb_reg = LGBMRegressor(random_state= 42, verbose = -100)
cat_reg = CatBoostRegressor(random_state= 42, verbose = False)

model_set = [lr_reg, tree_reg, rf_reg, xgb_reg, lgb_reg, cat_reg]

In [17]:
metricas_cv = {}

# Lista para almacenar los valores medios de las métricas
valores = []

# Iterar sobre cada modelo en el conjunto de modelos
for nombre, modelo in zip(model_names, model_set):
    print(modelo)
    # Calcular las métricas de validación cruzada para el modelo actual
    metricas_cv[nombre] = cross_val_score(modelo, X_train_encoded, y_train, cv=5, scoring="neg_mean_squared_error")
    
    # Calcular el valor medio de las métricas y añadirlo a la lista de valores
    valores.append(np.mean(metricas_cv[nombre]))

# Identificar el modelo ganador basado en el valor medio de las métricas
ganador = list(metricas_cv.keys())[np.argmax(valores)]

LinearRegression()
DecisionTreeRegressor(random_state=42)
RandomForestRegressor(random_state=42)
XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)
LGBMRegressor(random_state=42, verbose=-100)


In [18]:
for model_name, valores in metricas_cv.items():
    print(f"Modelo <{model_name}>, MAE_CV: {-np.mean(valores)}")
print(f"El ganador es {ganador}")

Modelo <Regresion Lineal>, MAE_CV: 69067948.67608929
Modelo <DecisionTree>, MAE_CV: 205921.39731346339
Modelo <Random Forest>, MAE_CV: 100146.55816968155
Modelo <XGBoost>, MAE_CV: 101366.35936131825
Modelo <LightGBM>, MAE_CV: 117698.80911356094
Modelo <CatBoost>, MAE_CV: 86843.41017753242
El ganador es CatBoost


In [19]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8],
    'iterations': [100, 200, 300]
}

# Inicializa un regresor CatBoost
catboost_reg = CatBoostRegressor()

# Configura el GridSearchCV con el regresor CatBoost, los hiperparámetros y la métrica de evaluación
grid_search = GridSearchCV(catboost_reg,
                           param_grid,
                           cv=5,  # Número de pliegues para la validación cruzada
                           scoring='neg_mean_absolute_error',  # Métrica de evaluación
                           n_jobs=-1
                          )

# Ajusta el GridSearchCV para encontrar el mejor modelo
grid_search.fit(X_train_encoded, y_train)

0:	learn: 689.6525090	total: 1.82ms	remaining: 545ms
1:	learn: 659.4164960	total: 3.67ms	remaining: 547ms
2:	learn: 624.1698949	total: 5.17ms	remaining: 512ms
3:	learn: 593.6912843	total: 6.51ms	remaining: 482ms
4:	learn: 566.9646254	total: 8ms	remaining: 472ms
5:	learn: 542.4606939	total: 9.46ms	remaining: 464ms
6:	learn: 518.9665903	total: 10.9ms	remaining: 457ms
7:	learn: 498.9405938	total: 12.4ms	remaining: 452ms
8:	learn: 480.7192610	total: 14ms	remaining: 454ms
9:	learn: 464.8284319	total: 15.3ms	remaining: 445ms
10:	learn: 449.3423447	total: 16.9ms	remaining: 443ms
11:	learn: 434.4616329	total: 18.5ms	remaining: 445ms
12:	learn: 422.6427977	total: 20.2ms	remaining: 447ms
13:	learn: 409.3647590	total: 21.9ms	remaining: 448ms
14:	learn: 397.8599015	total: 23.5ms	remaining: 447ms
15:	learn: 387.6245739	total: 25.1ms	remaining: 446ms
16:	learn: 377.6571218	total: 26.7ms	remaining: 445ms
17:	learn: 368.1882990	total: 29.4ms	remaining: 460ms
18:	learn: 361.4657005	total: 31ms	remainin

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x0000018362CB1930>,
             n_jobs=-1,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200, 300],
                         'learning_rate': [0.01, 0.05, 0.1]},
             scoring='neg_mean_absolute_error')

In [20]:
# Obtiene el mejor modelo encontrado
mejor_modelo_catboost = grid_search.best_estimator_

# Realiza predicciones en el conjunto de entrenamiento
predicciones_catboost_train = mejor_modelo_catboost.predict(X_train_encoded)

# Calcula el MAE en el conjunto de entrenamiento
mae_catboost_train = mean_absolute_error(y_train, predicciones_catboost_train)
print("MAE en el conjunto de entrenamiento usando CatBoost:", mae_catboost_train)

MAE en el conjunto de entrenamiento usando CatBoost: 81.1271281928112


In [21]:
print(X_test['Cpu'].head())  # Ver los primeros registros de la columna 'Cpu'

649    Intel Core i5 7300U 2.6GHz
761    Intel Core i5 7200U 2.5GHz
545    Intel Core i5 7200U 2.5GHz
367    Intel Core i5 8250U 1.6GHz
361    Intel Core i7 7500U 2.7GHz
Name: Cpu, dtype: object


In [22]:
print(X_test['Cpu'].isnull().sum())  # Contar valores nulos en la columna 'Cpu'

0


In [23]:
cpu_example = X_test['Cpu'].iloc[0]  # Tomar un ejemplo de CPU para probar la función
print(extract_cpu_info(cpu_example))  # Probar la función con un ejemplo de CPU

('Intel', 2.6, 'Core i5', '')


In [24]:
# Define la función para extraer información de la CPU
def extract_cpu_info(cpu):
    print("Valor de la CPU:", cpu)

    brand_pattern = r'(Intel|AMD)'
    speed_pattern = r'(\d+\.\d+)GHz'
    model_pattern = r'\b(?:Core [iI]\d+|Ryzen \d+)\b'
    series_pattern = r'(?:Intel|AMD)(.*?)(?:\b(?:Core [iI]\d+|Ryzen \d+|$)|\s\d+\.\d+GHz)'

    brand_match = re.search(brand_pattern, cpu)
    brand = brand_match.group(1) if brand_match else "Desconocida"
    print("Marca:", brand)

    speed_match = re.search(speed_pattern, cpu)
    speed = float(speed_match.group(1)) if speed_match else 1
    print("Velocidad:", speed)

    model_match = re.search(model_pattern, cpu)
    model = model_match.group() if model_match else "Desconocido"
    print("Modelo:", model)

    series_match = re.search(series_pattern, cpu)
    series = series_match.group(1).strip() if series_match else "Desconocida"
    print("Serie:", series)

    return brand, speed, model, series

# Aplica la función a X_test['Cpu'] y guarda los resultados
error_values = X_test['Cpu'].apply(extract_cpu_info)

# Filtra solo los valores que causaron problemas
problematic_values = error_values[error_values.apply(lambda x: isinstance(x, tuple) and len(x) != 4)]

# Imprime los valores problemáticos
print("Valores problemáticos en la columna 'Cpu':")
print(problematic_values)

Valor de la CPU: Intel Core i5 7300U 2.6GHz
Marca: Intel
Velocidad: 2.6
Modelo: Core i5
Serie: 
Valor de la CPU: Intel Core i5 7200U 2.5GHz
Marca: Intel
Velocidad: 2.5
Modelo: Core i5
Serie: 
Valor de la CPU: Intel Core i5 7200U 2.5GHz
Marca: Intel
Velocidad: 2.5
Modelo: Core i5
Serie: 
Valor de la CPU: Intel Core i5 8250U 1.6GHz
Marca: Intel
Velocidad: 1.6
Modelo: Core i5
Serie: 
Valor de la CPU: Intel Core i7 7500U 2.7GHz
Marca: Intel
Velocidad: 2.7
Modelo: Core i7
Serie: 
Valor de la CPU: Intel Core i3 6006U 2.0GHz
Marca: Intel
Velocidad: 2.0
Modelo: Core i3
Serie: 
Valor de la CPU: Intel Celeron Dual Core N3350 1.1GHz
Marca: Intel
Velocidad: 1.1
Modelo: Desconocido
Serie: Celeron Dual Core N3350
Valor de la CPU: Intel Celeron Dual Core N3060 1.6GHz
Marca: Intel
Velocidad: 1.6
Modelo: Desconocido
Serie: Celeron Dual Core N3060
Valor de la CPU: Intel Core i3 6006U 2GHz
Marca: Intel
Velocidad: 1
Modelo: Core i3
Serie: 
Valor de la CPU: Intel Core i5 8250U 1.6GHz
Marca: Intel
Velocidad

In [27]:
def extract_screen_resolution(df):
    def extract_resolution(screen_resolution):
        pattern = r'\b(\d{3,4})\s*x\s*(\d{3,4})\b'
        match = re.search(pattern, str(screen_resolution))
        if match:
            width = int(match.group(1))
            height = int(match.group(2))
            resolution = width * height
            return resolution
        else:
            return 1

    df['Resolution'] = df['ScreenResolution'].apply(extract_resolution)
    return df

# Define la función para extraer información de la CPU
def extract_cpu_info(df):
    def parse_cpu(cpu):
        try:
            cpu_str = str(cpu)
            print("Valor de CPU (antes de procesamiento):", cpu_str)
            brand_pattern = r'(Intel|AMD)'
            speed_pattern = r'(\d+\.\d+)GHz'
            model_pattern = r'\b(?:Core [iI]\d+|Ryzen \d+)\b'
            series_pattern = r'(?:Intel|AMD)(.*?)(?:\b(?:Core [iI]\d+|Ryzen \d+|$)|\s\d+\.\d+GHz)'

            brand_match = re.search(brand_pattern, cpu_str)
            brand = brand_match.group(1) if brand_match else "Desconocida"

            speed_match = re.search(speed_pattern, cpu_str)
            speed = float(speed_match.group(1)) if speed_match else -1

            model_match = re.search(model_pattern, cpu_str)
            model = model_match.group() if model_match else "Desconocido"

            series_match = re.search(series_pattern, cpu_str)
            series = series_match.group(1).strip() if series_match else "Desconocida"

            return brand, speed, model, series
        except Exception as e:
            print("Error durante la extracción de información de la CPU:", e)
            return "Desconocida", -1, "Desconocido", "Desconocida"

    cpu_info = df['Cpu'].apply(parse_cpu)
    df[['Cpu_Brand', 'Cpu_Speed', 'Cpu_Model', 'Cpu_Series']] = pd.DataFrame(cpu_info.tolist(), index=df.index)
    return df

# Define la función para extraer información de la memoria
def extract_memory_info(df):
    def parse_memory(memory):
        memory_pattern = r'(\d+)\s*(?:GB|T|TB)'
        keyword_pattern = r'HDD|SSD|Flash Storage|Hybrid'

        match = re.findall(memory_pattern, str(memory))
        if match:
            memory_value = int(match[0]) * 1024 if 'T' in memory else int(match[0])
        else:
            memory_value = 0

        keywords = re.findall(keyword_pattern, str(memory))
        storage_dict = {'HDD': 0, 'SSD': 0, 'Flash Storage': 0, 'Hybrid': 0}
        for keyword in keywords:
            storage_dict[keyword] = memory_value

        return storage_dict['HDD'], storage_dict['SSD'], storage_dict['Flash Storage'], storage_dict['Hybrid']

    memory_info = df['Memory'].apply(parse_memory)
    df[['HDD', 'SSD', 'Flash_Storage', 'Hybrid']] = pd.DataFrame(memory_info.tolist(), index=df.index)
    return df

# Define la función para eliminar outliers
def remove_outliers(df):
    numeric_cols = df.select_dtypes(include=np.number)
    Q1 = numeric_cols.quantile(0.25)
    Q3 = numeric_cols.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[~((numeric_cols < lower_bound) | (numeric_cols > upper_bound)).any(axis=1)]

# Define la función para aplicar One-Hot Encoding
def one_hot_encoding(df):
    return pd.get_dummies(df, drop_first=True)

# Crea un pipeline para aplicar todas las transformaciones
transformer_pipeline = Pipeline([
    ('screen_resolution_extraction', FunctionTransformer(extract_screen_resolution, validate=False)),
    ('cpu_info_extraction', FunctionTransformer(extract_cpu_info, validate=False)),
    ('memory_info_extraction', FunctionTransformer(extract_memory_info, validate=False)),
    ('outliers_removal', FunctionTransformer(remove_outliers, validate=False)),
    ('one_hot_encoding', FunctionTransformer(one_hot_encoding, validate=False)),
    ('scaler', StandardScaler())
])

# Aplica el pipeline a X_test
X_test_transformed = transformer_pipeline.fit_transform(X_test)

Valor de CPU (antes de procesamiento): Intel Core i5 7300U 2.6GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i5 8250U 1.6GHz
Valor de CPU (antes de procesamiento): Intel Core i7 7500U 2.7GHz
Valor de CPU (antes de procesamiento): Intel Core i3 6006U 2.0GHz
Valor de CPU (antes de procesamiento): Intel Celeron Dual Core N3350 1.1GHz
Valor de CPU (antes de procesamiento): Intel Celeron Dual Core N3060 1.6GHz
Valor de CPU (antes de procesamiento): Intel Core i3 6006U 2GHz
Valor de CPU (antes de procesamiento): Intel Core i5 8250U 1.6GHz
Valor de CPU (antes de procesamiento): Intel Celeron Dual Core N3060 1.6GHz
Valor de CPU (antes de procesamiento): Intel Core i7 6560U 2.2GHz
Valor de CPU (antes de procesamiento): Intel Core i5 7200U 2.5GHz
Valor de CPU (antes de procesamiento): Intel Core i7 8550U 1.8GHz
Valor de CPU (antes de procesamiento): Intel Cor

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

### 2. Dividir X_train, X_test, y_train, y_test

### 3. Crear y entrenar el modelo


### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de MAE.

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [ ]:
test = pd.read_csv("./data/test.csv")

In [ ]:
test_final = preprocess_data(test)

NameError: name 'preprocess_data' is not defined

In [ ]:
# Leer el archivo CSV
try:
    test = pd.read_csv("./data/test.csv")
    print("Archivo CSV cargado exitosamente.")
except FileNotFoundError:
    print("Error: El archivo 'test.csv' no se encuentra en la ruta especificada.")
except pd.errors.EmptyDataError:
    print("Error: El archivo 'test.csv' está vacío.")
except pd.errors.ParserError:
    print("Error: Hubo un problema al analizar el archivo 'test.csv'.")
except Exception as e:
    print(f"Error inesperado: {e}")

# Asegúrate de que la función preprocess_data esté definida antes de este punto
try:
    test_final = preprocess_data(test)
    print("Preprocesamiento completado exitosamente.")
except NameError:
    print("Error: La función 'preprocess_data' no está definida.")
except Exception as e:
    print(f"Error inesperado durante el preprocesamiento: {e}")

Archivo CSV cargado exitosamente.
Error inesperado durante el preprocesamiento: preprocess_data() missing 2 required positional arguments: 'cat_cols' and 'num_cols'


**¿Por qué puede dar error?** 

IMPORTANTE: APLICAR A ESTOS DATOS LO MISMO QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions = model.predict(X_pred)
predictions

NameError: name 'model' is not defined

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [ ]:
sample = pd.read_csv("Dataset/sample_submission.csv") # Esta es mi ruta del archivo, la vuestra puede ser otra

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeator(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.id.all() == sample.id.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("")


In [ ]:
chequeator(submission) # submission es el nombre que le habríamos puesto a nuestro .csv con los valores que me salieron en la predicción

You're ready to submit!
